# Tutorial for the transport-cba package

Created: 2020-02-12

Last update: 2023-07-30

This notebook provides an overview of the current fuctionality of the transport-cba Python module.

## 1. What is a CBA

[Cost-benefit analysis](https://en.wikipedia.org/wiki/Cost%E2%80%93benefit_analysis) (CBA) is a standard method for evaluating the meaningfulness of investments.  
An example of its recent use in the context of transport is the UK high speed railway (HS2) [more here](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/3651/hs2-economic-case-value-for-money.pdf).  

### 1.2. Costs and benefits of a new road

Generally, a new road has two types of costs:
1. capital expenditure (CAPEX), ie cost required to build the road
2. operation expenditure (OPEX), ie cost of servicing and maintaining the road

The benefits of using a road are of the following kinds:
1. Saved travel time. New road often allows higher speeds than the old one.
2. Saved operating costs. Eg oil and tyres, which depend on the number of kilometres passes.
3. Saved fuel. Depends on speed and number of kilometres passes.
4. Reduced accidents due to improved road parameters.
5. Reduced noise. Due to bypassing a town.
6. Reduced greenhouse gases and harmful emissions (eg PM2.5 or NOx) following from lowered fuel.

## 2. What is transport-cba

The aim of transport-cba is to remove the complexity and time 
spent on building an overly complex and unwieldy Excel file by defining
a clear structure of inputs and automatising the computation.
 
As a result, this frees up time and expenses spent on manual tasks.
The freed resources can be focused on most important problems
of road planning, among others:
* fast and consistent evaluation of many projects in the country and consequent prioritisation
* evaluation of several scenarios of the growth of intensities
* exploration of different building scenarios (eg change in build time) 
* and variation of BCR with capital expenditures
* exploration of different road routing
 
Although Excel is an extremely useful tool, one of the key 
functionalities that it cannot provide is consistency and
reproducibility. It is not only easy to make an error in the formula,
but also deliberately hide a computation and mislead the decision makers.
Python, like any programming language, enables im principle to minimise this risk.

### 2.1. Inputs

transport-cba uses project inputs and parameters.

There are four types of inputs amounting to six tables:
1. Parameters of old and new road sections.
2. Capital expenditures (CAPEX).
3. Vehicle intensities in variants 0 and 1.
4. Vehicle velocities in variants 0 and 1.

There are many kinds of parameter inputs related to the costs and benefits. 
Each is stored in a table in the `files` directory.

#### Examples of road parameters

In [1]:
from transport_cba.sample_projects import load_sample_bypass

byp = load_sample_bypass()

In [2]:
byp["RP"]

,name,variant_0,variant_1,length,length_bridges,length_tunnels,category,lanes,environment,width,layout,toll_sections
id_section,,,,,,,,,,,,
0,entrance,1,1,0.5,0.0,0,standard,2,extravilan,9.5,wide,1
1,city,1,1,4.0,0.0,0,standard,2,intravilan,9.5,narrow,1
2,exit,1,1,0.4,0.0,0,standard,2,extravilan,9.5,wide,1
3,bypass,0,1,3.9,0.1,0,standard,2,extravilan,11.5,wide,1


Each road section is defined by an ID and has a name for easier identification.

A section can be used in variant 0, in which case it already exists. 
Sections in variant 1 are the ones to build.

The rest of the parameters of any section:
1. Length
2. Length of the bridges important for the calculation of operation cost
3. Length of the tunnels, same
4. Category (standard or motorway) to compute unit cost of maintenance
5. Environment (intravilan/extravilan), important for the impact of emissions
6. Number of lanes
7. Width
8. Layout important for the accident rates
9. Toll sections, important for the operation cost of the toll system

In [3]:
byp["C_fin"]

,,2020,2021,2022
item,category,,,
land,preparation,300000,300000,300000
pavements,construction,14000000,14000000,14000000
bridges,construction,1500000,1500000,1500000
tunnels,construction,0,0,0
buildings,construction,0,0,0
slope_stabilisation,construction,0,0,0
retaining_walls,construction,0,0,0
noise_barriers,construction,300000,300000,300000
safety_features,construction,0,0,0


Capital expenditures (CAPEX) is a dataframe of investment in every of the building years.
The investment items fall into several categories.

In [4]:
byp["I0"]

2020   2021   2022   2023   2024   2025   2026   2027  \
id_section vehicle                                                           
0          bus         20     20     20     20     20     20     20     20   
           car       9000   9090   9181   9273   9366   9460   9555   9651   
           hgv       1000   1010   1020   1030   1040   1050   1061   1072   
           lgv          0      0      0      0      0      0      0      0   
1          bus         20     20     20     20     20     20     20     20   
           car      10000  10100  10201  10303  10406  10510  10615  10721   
           hgv       1000   1010   1020   1030   1040   1050   1061   1072   
           lgv          0      0      0      0      0      0      0      0   
2          bus         20     20     20     20     20     20     20     20   
           car       8000   8080   8161   8243   8325   8408   8492   8577   
           hgv       1000   1010   1020   1030   1040   1050   1061   1072   
           lgv          0      0      0      0      0      0      0      0   

                     2028   2029  ...   2040   2041   2042   2043   2044  \
id_section vehicle                ...                                      
0          bus         20     20  ...     20     20     20     20     20   
           car       9748   9845  ...  10982  11092  11203  11315  11428   
           hgv       1083   1094  ...   1220   1232   1244   1256   1269   
           lgv          0      0  ...      0      0      0      0      0   
1          bus         20     20  ...     20     20     20     20     20   
           car      10828  10936  ...  12201  12323  12446  12570  12696   
           hgv       1083   1094  ...   1220   1232   1244   1256   1269   
           lgv          0      0  ...      0      0      0      0      0   
2          bus         20     20  ...     20     20     20     20     20   
           car       8663   8750  ...   9763   9861   9960  10060  10161   
           hgv       1083   1094  ...   1220   1232   1244   1256   1269   
           lgv          0      0  ...      0      0      0      0      0   

                     2045   2046   2047   2048   2049  
id_section vehicle                                     
0          bus         20     20     20     20     20  
           car      11542  11657  11774  11892  12011  
           hgv       1282   1295   1308   1321   1334  
           lgv          0      0      0      0      0  
1          bus         20     20     20     20     20  
           car      12823  12951  13081  13212  13344  
           hgv       1282   1295   1308   1321   1334  
           lgv          0      0      0      0      0  
2          bus         20     20     20     20     20  
           car      10263  10366  10470  10575  10681  
           hgv       1282   1295   1308   1321   1334  
           lgv          0      0      0      0      0  

[12 rows x 30 columns]

Intensities in variants 0 and 1 are the number of vehicles in both directions in one day, 
divided by vehicle type and road section.

On new sections, intensities should be trivially zero before the section is built.

In [5]:
byp["V0"]

2020  2021  2022  2023  2024  2025  2026  2027  2028  \
id_section vehicle                                                         
0          bus        70    70    70    75    75    75    75    75    75   
           car        80    80    80    80    80    80    80    80    80   
           hgv        70    70    70    70    70    70    70    70    70   
           lgv        80    80    80    80    80    80    80    80    80   
1          bus        50    50    50    50    50    50    50    50    50   
           car        50    50    50    50    50    50    50    50    50   
           hgv        50    50    50    50    50    50    50    50    50   
           lgv        50    50    50    50    50    50    50    50    50   
2          bus        70    70    70    75    75    75    75    75    75   
           car        80    80    80    80    80    80    80    80    80   
           hgv        70    70    70    70    70    70    70    70    70   
           lgv        80    80    80    80    80    80    80    80    80   

                    2029  ...  2040  2041  2042  2043  2044  2045  2046  2047  \
id_section vehicle        ...                                                   
0          bus        75  ...    75    75    75    75    75    75    75    75   
           car        80  ...    80    80    80    80    80    80    80    80   
           hgv        70  ...    70    70    70    70    70    70    70    70   
           lgv        80  ...    80    80    80    80    80    80    80    80   
1          bus        50  ...    50    50    50    50    50    50    50    50   
           car        50  ...    50    50    50    50    50    50    50    50   
           hgv        50  ...    50    50    50    50    50    50    50    50   
           lgv        50  ...    50    50    50    50    50    50    50    50   
2          bus        75  ...    75    75    75    75    75    75    75    75   
           car        80  ...    80    80    80    80    80    80    80    80   
           hgv        70  ...    70    70    70    70    70    70    70    70   
           lgv        80  ...    80    80    80    80    80    80    80    80   

                    2048  2049  
id_section vehicle              
0          bus        75    75  
           car        80    80  
           hgv        70    70  
           lgv        80    80  
1          bus        50    50  
           car        50    50  
           hgv        50    50  
           lgv        50    50  
2          bus        75    75  
           car        80    80  
           hgv        70    70  
           lgv        80    80  

[12 rows x 30 columns]

Velocities in variants 0 and 1 differ with vehicles and road sections,
due to different speed limits or the nature of movement (trucks are slower than cars).

Velocities in variants 0 and 1 come from an external source, 
most likely a transport model. This can account for more sophisticated features
such as saturation due to overly high intensities.

### 2.3. Computation

Here is a simple code:

In [6]:
from transport_cba import RoadCBA

yr_start = 2020
price_level = 2020
country = "svk"

rcba = RoadCBA(yr_start, price_level, country)
rcba.load_parameters(source="default")
rcba.read_project_inputs(byp["RP"], byp["C_fin"], byp["I0"], byp["I1"], byp["V0"], byp["V1"])

In [7]:
rcba.economic_analysis()

ENPV: 3.22 M EUR
ERR : 5.60 %
BCR : 1.07
Time: 1.14 s.


The output of the economic analysis is:
* net present value in millions of EUR (or other currency)
* economic rate of return as a percentage
* benefit to cost ratio (BCR)

### 2.4. Outputs

It is possible to print the values of each of the benefits and costs.

The economic present value of each benefit and cost is stored in `df_enpv`.

In [8]:
rcba.df_enpv

value
type    item                
cost    capex   -41955414.40
        opex     -2077088.92
benefit res_val   1689630.93
        vtts     21128101.22
        voc       9083415.88
        acc       3237916.19
        fuel      2360650.55
        gg         588108.43
        em        3481194.67
        noise     5681515.11

The dataframe storing all economic benefits and costs with time is stored in `df_eco`.

In [9]:
rcba.df_eco

2020        2021        2022        2023        2024  \
type    item                                                                  
cost    capex   -14672750.0 -14672750.0 -14672750.0       -0.00       -0.00   
        opex           -0.0        -0.0        -0.0   -87935.90   -87935.90   
benefit res_val         0.0         0.0         0.0        0.00        0.00   
        vtts            0.0         0.0         0.0  1444583.36  1458568.83   
        voc             0.0         0.0         0.0   610608.50   616696.70   
        acc             0.0         0.0         0.0   221804.96   223865.98   
        fuel            0.0         0.0         0.0   159101.27   160688.78   
        gg              0.0         0.0         0.0    32658.65    33852.28   
        em              0.0         0.0         0.0   236936.03   239133.87   
        noise           0.0         0.0         0.0   388875.64   392481.05   

                       2025        2026        2027        2028        2029  \
type    item                                                                  
cost    capex         -0.00       -0.00       -0.00       -0.00       -0.00   
        opex      -87935.90   -87935.90   -87935.90   -87935.90   -87935.90   
benefit res_val        0.00        0.00        0.00        0.00        0.00   
        vtts     1470380.37  1485503.93  1499604.62  1514935.67  1529171.11   
        voc       622598.75   630565.97   638368.94   646418.29   654248.63   
        acc       225690.04   227909.12   230055.21   232316.82   234482.24   
        fuel      162232.92   164194.77   166118.17   168103.58   170038.23   
        gg         35054.71    36368.36    37694.94    39057.06    40428.43   
        em        241144.56   243880.91   246596.84   249343.45   252064.40   
        noise     395809.68   399700.47   403585.33   407493.81   411396.34   

                 ...        2040        2041        2042        2043  \
type    item     ...                                                   
cost    capex    ...       -0.00       -0.00       -0.00       -0.00   
        opex     ...   -87935.90   -87935.90   -87935.90   -87935.90   
benefit res_val  ...        0.00        0.00        0.00        0.00   
        vtts     ...  1689644.29  1706452.66  1723439.16  1740082.15   
        voc      ...   736454.66   743802.11   751182.41   758584.61   
        acc      ...   258636.04   261225.61   263842.87   266283.67   
        fuel     ...   190991.40   192899.86   194820.81   196746.69   
        gg       ...    51616.23    52652.71    53703.64    54766.28   
        em       ...   279427.06   282231.65   285049.55   287668.97   
        noise    ...   453938.87   458529.18   463148.16   467441.33   

                       2044        2045        2046        2047        2048  \
type    item                                                                  
cost    capex         -0.00       -0.00       -0.00       -0.00       -0.00   
        opex      -87935.90   -87935.90  -929375.72   -87935.90   -87935.90   
benefit res_val        0.00        0.00        0.00        0.00        0.00   
        vtts     1757265.31  1774621.09  1792026.52  1809610.08  1825947.69   
        voc       766019.66   773482.09   780993.79   788538.34   796137.63   
        acc       268935.95   271613.61   274311.88   277037.83   279605.57   
        fuel      198685.04   200634.64   202595.33   204568.49   206559.06   
        gg         55841.88    56931.88    58035.47    59152.29    60285.52   
        em        290500.26   293344.34   296193.07   299055.10   301722.06   
        noise     472085.76   476758.83   481431.95   486133.72   490521.95   

                       2049  
type    item                 
cost    capex         -0.00  
        opex      -87935.90  
benefit res_val  6954750.00  
        vtts     1843218.20  
        voc       802284.97  
        acc       282277.20  
        fuel      208272.40  
        gg         61344.61  
        em 

These frames can consequently be stored in Excel format for further use.

## 3. The logic and functionality of RoadCBA object

The results mentioned above are computed in several methods
that are presented here.

In [10]:
del rcba
rcba = RoadCBA(yr_start, price_level, "svk", verbose=True)

At the beginning, the parameters and the project inputs are read.  
Parameters are read into the dictionary `df_raw` containing the relevant dataframes.

In [11]:
rcba.load_parameters()
rcba.read_project_inputs(byp["RP"], byp["C_fin"], byp["I0"], byp["I1"], byp["V0"], byp["V1"])

# for reading whole excel files use this with the name of the Excel sheet:
# rcba.read_project_inputs_xls()

Reading CBA parameters...
Reading project inputs from df...


In [12]:
# content of the `df_raw` dictionary
rcba.df_raw.keys()

dict_keys(['c_op', 'toll_op', 'res_val', 'c_fuel', 'conv_fac', 'occ_p', 'occ_f', 'r_tp', 'vtts', 'voc', 'fuel_coeffs', 'r_fuel', 'r_acc', 'c_acc', 'r_gg', 'c_gg', 'r_em', 'c_em', 'noise'])

Then, parameters need to be prepared.
This includes creating vectors of CPI and GDP growth for all the relevant years,
adjusting the price level to the given year
and other necessary modifications (eg merging of several parameter tables).

In [13]:
rcba.prepare_parameters()

Adjusting CPI...
Cleaning parameters...
    Cleaning: c_op
    Cleaning: toll_op
    Cleaning: res_val
    Cleaning: c_fuel
    Cleaning: conv_fac
    Cleaning: occ_p
    Cleaning: occ_f
    Cleaning: r_tp
    Cleaning: vtts
    Cleaning: voc
    Cleaning: fuel_coeffs
    Cleaning: r_fuel
    Cleaning: r_acc
    Cleaning: c_acc
    Cleaning: r_gg
    Cleaning: c_gg
    Cleaning: r_em
    Cleaning: c_em
    Cleaning: noise
Adjusting price level...
    Adjusting: c_op
    Adjusting: toll_op
    Adjusting: vtts
    Adjusting: voc
    Adjusting: c_fuel
    Adjusting: c_acc
    Adjusting: c_gg
    Adjusting: c_em
    Adjusting: noise
Wrangling parameters...
Averaging VTTS over distance type.


Then, CAPEX, OPEX and residual value matrices are computed.

The CAPEX matrices for financial and economic analysis is stored in the `C_fin` and `C_eco` dataframes respectively.

### CAPEX

In [14]:
rcba.compute_capex()
rcba.C_fin

Computing CAPEX...


,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
item,,,,,,,,,,,,,,,,,,,,,
land,300000,300000,300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pavements,14000000,14000000,14000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bridges,1500000,1500000,1500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tunnels,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
buildings,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
slope_stabilisation,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
retaining_walls,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
noise_barriers,300000,300000,300000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
safety_features,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### OPEX

Before computing OPEX, matrices of unit cost must be created.

In [15]:
rcba._create_unit_cost_matrix()
rcba._create_unit_cost_opex_mask()

Creating time matrices for benefits...
    Creating: c_op
    Creating: toll_op
    Creating: vtts
    Creating: voc
    Creating: c_fuel
    Creating: c_acc
    Creating: c_em
    Creating: noise


In [16]:
rcba.UC.keys()

dict_keys(['c_op', 'toll_op', 'vtts', 'voc', 'c_fuel', 'c_acc', 'c_em', 'noise', 'c_gg'])

Now OPEX can be computed. It is different variants 0 and 1, 
so two separate dataframes, `O0` and `O1`, are created.

Both are in versions for financial and economic analysis 
with `_fin` and `_eco` suffices respectively.

In [17]:
rcba.compute_opex()
rcba.O0_eco

Computing OPEX...


2020        2021  \
id_section item              operation_type                           
0          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine          9080.3375   9080.3375   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
1          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine         72642.7000  72642.7000   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
2          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine          7264.2700   7264.2700   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   

                                                   2022        2023  \
id_section item              operation_type                           
0          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine          9080.3375   9080.3375   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
1          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine         72642.7000  72642.7000   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
2          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine          7264.2700   7264.2700   
           pavements_renewal periodic            0.0000      0.0000   
           pavements_repair  periodic            0.0000      0.0000   
           road_marking      periodic            0.0000      0.0000   
           tunnels           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   

                                                   2024        2025  \
id_section item              operation_type                           
0          bridges           periodic            0.0000      0.0000   
                             routine             0.0000      0.0000   
           pavements         routine          9080.3375   9080.3375   
  

### Residual value

The next step is residual value, ie the value of the buildings at the end of the economic period.

In [18]:
rcba.compute_residual_value()
rcba.RV_eco

Computing residual value...


,lifetime,replacement_cost_ratio,op_period,replace,rem_ratio,value,res_value
item,,,,,,,
land,inf,1.00,27,0,1.00,900000.0,900000.0
pavements,30.0,0.75,27,0,0.10,37800000.0,2835000.0
pavements_concrete,30.0,0.75,27,0,0.10,NaN,NaN
bridges,100.0,1.00,27,0,0.73,4050000.0,2956500.0
tunnels,100.0,1.00,27,0,0.73,0.0,0.0
buildings,50.0,1.00,27,0,0.46,0.0,0.0
slope_stabilisation,20.0,0.75,27,1,0.65,0.0,0.0
retaining_walls,30.0,0.75,27,0,0.10,0.0,0.0
noise_barriers,20.0,0.50,27,1,0.65,810000.0,263250.0


### Benefits

The first benefit is the value of travel time saved
in variant 1 compared with variant 0.

First the travel time needs to be computed.

In [19]:
rcba._create_length_matrix()

Creating length matrix...


In [20]:
rcba._compute_travel_time_matrix()

Creating matrices of travel times...


In [21]:
rcba._compute_vtts()
rcba.NB["vtts"]

    Computing VTTS...


2020    0.000000e+00
2021    0.000000e+00
2022    0.000000e+00
2023    1.444583e+06
2024    1.458569e+06
2025    1.470380e+06
2026    1.485504e+06
2027    1.499605e+06
2028    1.514936e+06
2029    1.529171e+06
2030    1.543284e+06
2031    1.556821e+06
2032    1.569858e+06
2033    1.583901e+06
2034    1.596684e+06
2035    1.610853e+06
2036    1.626314e+06
2037    1.643071e+06
2038    1.658391e+06
2039    1.674181e+06
2040    1.689644e+06
2041    1.706453e+06
2042    1.723439e+06
2043    1.740082e+06
2044    1.757265e+06
2045    1.774621e+06
2046    1.792027e+06
2047    1.809610e+06
2048    1.825948e+06
2049    1.843218e+06
dtype: float64

Computing vehicle operating costs:

In [22]:
rcba._compute_voc()

    Computing VOC...


Computing accident costs:

In [23]:
rcba._compute_accidents()

    Computing accidents...


Before computing the remaining benefits, 
we must calculate the fuel consumption by road sections:

In [24]:
rcba._create_fuel_ratio_matrix()

Creating matrix of fuel ratios by vehicle...


In [25]:
rcba._compute_fuel_consumption()

    Computing fuel consumption...


Now the cost of fuel, emissions and greenhouse gases can be computed:

In [26]:
rcba._compute_fuel_cost()

    Computing fuel cost...


In [27]:
rcba._compute_emissions()

    Computing emissions...


In [28]:
rcba._compute_greenhouse()

    Computing greenhouse gases...


Finally, the economic benefit of reduced noise is computed:

In [29]:
rcba._compute_noise()

    Computing noise...


### Economic analysis

Finally, one can finalise the economic analysis by computing the relevant indicators: ENPV, ERR and BCR.

In [30]:
rcba.compute_economic_indicators()


Computing ENPV, ERR, BCR...
ENPV: 3.22 M EUR
ERR : 5.60 %
BCR : 1.07


The resulting matrix of cost and benefits broken down by years is stored in `df_eco`:

In [31]:
rcba.df_eco

2020        2021        2022        2023        2024  \
type    item                                                                  
cost    capex   -14672750.0 -14672750.0 -14672750.0       -0.00       -0.00   
        opex           -0.0        -0.0        -0.0   -87935.90   -87935.90   
benefit res_val         0.0         0.0         0.0        0.00        0.00   
        vtts            0.0         0.0         0.0  1444583.36  1458568.83   
        voc             0.0         0.0         0.0   610608.50   616696.70   
        acc             0.0         0.0         0.0   221804.96   223865.98   
        fuel            0.0         0.0         0.0   159101.27   160688.78   
        em              0.0         0.0         0.0   236936.03   239133.87   
        gg              0.0         0.0         0.0    32658.65    33852.28   
        noise           0.0         0.0         0.0   388875.64   392481.05   

                       2025        2026        2027        2028        2029  \
type    item                                                                  
cost    capex         -0.00       -0.00       -0.00       -0.00       -0.00   
        opex      -87935.90   -87935.90   -87935.90   -87935.90   -87935.90   
benefit res_val        0.00        0.00        0.00        0.00        0.00   
        vtts     1470380.37  1485503.93  1499604.62  1514935.67  1529171.11   
        voc       622598.75   630565.97   638368.94   646418.29   654248.63   
        acc       225690.04   227909.12   230055.21   232316.82   234482.24   
        fuel      162232.92   164194.77   166118.17   168103.58   170038.23   
        em        241144.56   243880.91   246596.84   249343.45   252064.40   
        gg         35054.71    36368.36    37694.94    39057.06    40428.43   
        noise     395809.68   399700.47   403585.33   407493.81   411396.34   

                 ...        2040        2041        2042        2043  \
type    item     ...                                                   
cost    capex    ...       -0.00       -0.00       -0.00       -0.00   
        opex     ...   -87935.90   -87935.90   -87935.90   -87935.90   
benefit res_val  ...        0.00        0.00        0.00        0.00   
        vtts     ...  1689644.29  1706452.66  1723439.16  1740082.15   
        voc      ...   736454.66   743802.11   751182.41   758584.61   
        acc      ...   258636.04   261225.61   263842.87   266283.67   
        fuel     ...   190991.40   192899.86   194820.81   196746.69   
        em       ...   279427.06   282231.65   285049.55   287668.97   
        gg       ...    51616.23    52652.71    53703.64    54766.28   
        noise    ...   453938.87   458529.18   463148.16   467441.33   

                       2044        2045        2046        2047        2048  \
type    item                                                                  
cost    capex         -0.00       -0.00       -0.00       -0.00       -0.00   
        opex      -87935.90   -87935.90  -929375.72   -87935.90   -87935.90   
benefit res_val        0.00        0.00        0.00        0.00        0.00   
        vtts     1757265.31  1774621.09  1792026.52  1809610.08  1825947.69   
        voc       766019.66   773482.09   780993.79   788538.34   796137.63   
        acc       268935.95   271613.61   274311.88   277037.83   279605.57   
        fuel      198685.04   200634.64   202595.33   204568.49   206559.06   
        em        290500.26   293344.34   296193.07   299055.10   301722.06   
        gg         55841.88    56931.88    58035.47    59152.29    60285.52   
        noise     472085.76   476758.83   481431.95   486133.72   490521.95   

                       2049  
type    item                 
cost    capex         -0.00  
        opex      -87935.90  
benefit res_val  6954750.00  
        vtts     1843218.20  
        voc       802284.97  
        acc       282277.20  
        fuel      208272.40  
        em        304247.30  
        gg 

The summary of present values by costs and benefits is stored in `df_enpv`:

In [32]:
rcba.df_enpv

value
type    item                
cost    capex   -41955414.40
        opex     -2077088.92
benefit res_val   1689630.93
        vtts     21128101.22
        voc       9083415.88
        acc       3237916.19
        fuel      2360650.55
        em        3481194.67
        gg         588108.43
        noise     5681515.11